In [1]:
%pip3 install atari_py
%pip3 install Box2D
%pip3 install box2d-py
%pip3 install "stable-baselines3[extra]>=2.0.0a4"
%pip3 install gym
%pip3 install opencv-python
%sudo apt-get install -y swig build-essential python3-dev
%sudo apt-get install -y swig build-essential python3-dev
%pip install gymnasium[box2d]
%pip install box2d-py
%pip install box2d-py-manylinux1
%pip install git+https://github.com/pybox2d/pybox2d

Defaulting to user installation because normal site-packages is not writeable
  Using cached atari_py-0.2.9-cp311-cp311-linux_x86_64.whl
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.9/427.9 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [16 lines of output]
      Using setuptools (version 69.1.1).
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-311
      creating build/lib.linux-x86_64-cpython-311/Box2D
      copying library/Box2D/__init__.py -> build/lib.linux-x86_64-cpython-311/Box2D
      copying library/Box2D/Box2D.py -> build/lib.linux-x86_64-cpython-311/Box2D
      creating build/lib.linux-x86_64-cpython-311/Box2D/b2
      copying library/Box2D/b2/__init__.py -> buil

In [1]:
import torch as th
import cv2
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import StopTrainingOnRewardThreshold, EvalCallback
import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
class AtariGames:
    def __init__(self, env_path, render_mode, path_save_best, path_save, video_path):
        self.env_path = env_path
        self.render_mode = render_mode
        self.path_save_best = path_save_best
        self.path_save = path_save
        self.video_path = video_path

    def make_env(self, which_env):
        self.which_env = which_env
        env = gym.make(self.env_path[self.which_env], render_mode=self.render_mode)
        self.env = DummyVecEnv([lambda: env])
        
    def train_model(self, total_timesteps, episodes, policy_kwargs, reward_threshold, eval_freq):
        stop_callback_func = StopTrainingOnRewardThreshold(reward_threshold=reward_threshold, verbose=1)
        eval_callback = EvalCallback(self.env,
                                      callback_on_new_best=stop_callback_func,
                                      eval_freq=eval_freq,
                                      best_model_save_path=self.path_save_best[0],
                                      verbose=1)


        self.model = PPO("MlpPolicy", self.env,  verbose=1, policy_kwargs=policy_kwargs)
        self.model.learn(total_timesteps=total_timesteps, callback=eval_callback)
        evaluate_policy(self.model, self.env, n_eval_episodes=episodes, render=True)
        
        self.model.save(self.path_save[self.which_env])
        del self.model
        
    def test_model(self, episodes, fps):
        model = PPO.load(self.path_save[self.which_env], env=self.env)
        observation= self.env.reset()
        frame_size = (self.env.render().shape[1], self.env.render().shape[0])

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(self.video_path[self.which_env],  fourcc, fps, frame_size)
        for i in range(1, episodes+1):
            observation = self.env.reset()
            score = 0
            done = False

            while not done:
                frame = self.env.render()
                action, _ = model.predict(observation)
                observation, reward, done, _ = self.env.step(action)
                frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                video_writer.write(frame_bgr)
                score+=reward

            print(f"Episode: {i}, score: {score}")
        video_writer.release()

In [ ]:
env = ('CartPole-v1', "LunarLander-v2", "CarRacing-v2")

# Render mode
render_mode = "rgb_array"

# Path to the best reward
path_save_best = ("CartPole-v1-BEST", "LunarLander-v2-BEST", "CarRacing-v4")

# Path to model
path_save = ("cart_pole_model.zip", "lunar_landler.zip", "car_racing.zip")

# Path to Video
video_path = ("CartPole-video.mp4", "LunarLander-video.mp4", "CarRacing-video.mp4")

policy_kwargs = dict(pi=[64, 128, 128, 64], vf=[64, 128, 128, 64])
act_fn=th.nn.ReLU
custom_policy_kwargs = dict(net_arch=policy_kwargs, activation_fn=act_fn)

games = AtariGames(env, render_mode, path_save_best, path_save, video_path)

In [ ]:
# CartPole game
games.make_env(0)
games.train_model(total_timesteps=100000, episodes=10, policy_kwargs=custom_policy_kwargs, reward_threshold=500, eval_freq= 18000)
games.test_model(episodes=10, fps=35)

# result below code

In [ ]:
# LunarLander game
games.make_env(1)
games.train_model(total_timesteps=100000, episodes=10, policy_kwargs=custom_policy_kwargs, reward_threshold=300, eval_freq= 18000)
games.test_model(episodes=10, fps=35)

# result below code

In [ ]:
# CarRacing game
games.make_env(2)
games.train_model(total_timesteps=100000, episodes=10, policy_kwargs=custom_policy_kwargs, reward_threshold=500, eval_freq= 18000)
games.test_model(episodes=10, fps=35)

# result below code